MANGENI HEARTY MARTHA
 Access no. B29839 
 Registration no. S24B13/014

Question 1 : Encapsulation in Daily Reality


Encapsulation refers to the bundling of data (attributes) and the methods (functions) that operate on the data into a single unit (a class). It's a mechanism that restricts direct access to some of an object's components.

While

Data Hiding is the practice of restricting direct access to some of an object's components for example using private attributes to prevent external code from depending on internal representation.

Qtn 1.2

In a family setting where household finances are managed jointly, encapsulation can be used to prevent misuse of funds by creating a single "HouseholdBudget" system. Only the  head of the family, designated parent or the treasurer can add expenses and update the budget. Other family members can request funds or view assigned allowances but cannot directly change totals. This prevents accidental or malicious modification of the total budget and ensures all changes go through validated processes For example needing a note or purpose for each withdrawal.


ASCII UML Diagram
```
+-------------------+
| HouseholdBudget   |
+-------------------+
| - _total_budget   |
| - _owner_name     |
+-------------------+
| + get_total()     |
| + add_expense()   |
| + add_income()    |
+-------------------+
```

In [1]:
# Code Implementation
# Implementing the HouseholdBudget class using getters/setters 
import time

class HouseholdBudget:
    # A simple household budget class demonstrating encapsulation.
    
    def __init__(self, owner_name, initial_amount=0.0):
        # Private attribute 
        self.__owner_name = owner_name
        self.__total_budget = float(initial_amount)

    # Getter for owner name
    def get_owner_name(self):
        return self.__owner_name

    # Getter for total budget
    def get_total_budget(self):
        return self.__total_budget

    # Setter for owner name (with simple validation)
    def set_owner_name(self, name):
        if not isinstance(name, str) or not name.strip():
            raise ValueError("Owner name must be a non-empty string")
        self.__owner_name = name.strip()

    # Add income to the budget
    def add_income(self, amount, note=''):
        # Validation: income must be positive and reasonable. This prevents
        # accidental negative entries which would reduce the budget.
        if amount <= 0:
            raise ValueError("Income amount must be positive")
        if amount > 10_000_000:  # district-specific sanity upper bound
            raise ValueError("Income amount unreasonably large for household")
        self.__total_budget += float(amount)
        return {'timestamp': time.ctime(), 'change': amount, 'note': note}

    # Add expense (reduce budget)
    def add_expense(self, amount, note=''):
        # Validation: expense must be positive and not exceed current budget
        if amount <= 0:
            raise ValueError("Expense amount must be positive")
        if amount > self.__total_budget:
            raise ValueError("Insufficient funds for this expense")
        self.__total_budget -= float(amount)
        return {'timestamp': time.ctime(), 'change': -amount, 'note': note}

    # A summary
    def summary(self):
        return f"Owner: {self.__owner_name} | Total Budget: UGX {self.__total_budget:.2f}"

# Demonstration: one valid run and one invalid run
print("--- Valid run ---")
hb = HouseholdBudget("Martha's Family", initial_amount=500000.0)
print(hb.summary())
print(hb.add_income(100000, note='Sale of crops'))
print(hb.summary())

print("\n--- Invalid run (expense exceeds budget) ---")
try:
    hb.add_expense(1_000_000, note='Attempt large purchase')
except Exception as e:
    print('Caught Exception:', e)


--- Valid run ---
Owner: Martha's Family | Total Budget: UGX 500000.00
{'timestamp': 'Wed Oct  8 20:15:09 2025', 'change': 100000, 'note': 'Sale of crops'}
Owner: Martha's Family | Total Budget: UGX 600000.00

--- Invalid run (expense exceeds budget) ---
Caught Exception: Insufficient funds for this expense


QUESTION 2: ALPHA MIS SIMULATION

Alpha MIS Brief

In a university management system like Alpha MIS, encapsulation helps by containing student data (personal details, course registrations, fees) inside classes such as `Student`, `Course`, and `FeesRecord`. The system exposes methods for registering courses or paying fees while preventing other parts of the system from directly manipulating sensitive fields like fee balances or changeable IDs. This ensures validations (credit limits, fee thresholds) are applied uniformly and audit trails are produced for important updates.


In [5]:
# Question 2 — Code Implementation
# Implementing a small part of Alpha MIS: CourseRegistration with encapsulation
import time

class CourseRegistration:
    #Represents course registration for a single student in UCU context.
    #Demonstrates public, _protected, and __private attributes.
    #Includes validation: credit limit and fees threshold.

    def __init__(self, access_number, faculty, max_credits=24):
        # public
        self.access_number = access_number
        # protected 
        self._faculty = faculty
        # private 
        self.__registered_courses = {}  # code -> credits
        self.__current_credits = 0
        self.__fee_balance = 0.0
        self.__max_credits = max_credits

    # public method to register a course
    def register_course(self, course_code, credits, course_fee):
        # Validation 1: credit limit (district / faculty rule)
        if credits <= 0:
            raise ValueError("Course credits must be positive")
        if self.__current_credits + credits > self.__max_credits:
            raise ValueError(
                f"Cannot register {course_code}: exceeds max credit limit ({self.__max_credits})"
            )
        # Validation 2: fees threshold before registering too many courses
        projected_balance = self.__fee_balance + float(course_fee)
        if projected_balance > 2_000_000:  # example threshold for UCU context
            raise ValueError("Fee balance would exceed allowed threshold; clear dues first")
        # If validations pass, register course
        self.__registered_courses[course_code] = credits
        self.__current_credits += credits
        self.__fee_balance = projected_balance
        return True

    # public method to pay fees
    def pay_fees(self, amount):
        if amount <= 0:
            raise ValueError("Payment must be positive")
        self.__fee_balance -= float(amount)
        if self.__fee_balance < 0:
            self.__fee_balance = 0.0
        return {'timestamp': time.ctime(), 'paid': amount}

    # protected helper to list courses (not intended for external modification)
    def _list_courses(self):
        return dict(self.__registered_courses)

    # summary/report method printing Access Number, faculty and timestamp
    def report(self):
        ts = time.ctime()
        return f"Access: {self.access_number} | Faculty: {self._faculty} | Time: {ts} | Credits: {self.__current_credits} | Balance UGX {self.__fee_balance:.2f}"

# Demonstration
cr = CourseRegistration('b29839', 'Faculty of Computing and IT', max_credits=24)
print('Initial report:')
print(cr.report())

print('\nRegistering CSC2105 (3 credits) — should succeed')
cr.register_course('CSC2105', 3, 150000)
print(cr.report())

print('\nAttempting to register a large load that exceeds credits — should raise')
try:
    cr.register_course('BIGLOAD', 30, 500000)
except Exception as e:
    print('Caught Exception:', e)

print('\nAttempting to register courses that push fee balance beyond threshold — should raise')
try:
    # multiple registrations to increase balance
    cr.register_course('CSC3001', 3, 1_900_000)
except Exception as e:
    print('Caught Exception:', e)

print('\nPaying part of the fees (100000)')
print(cr.pay_fees(100000))
print(cr.report())


Initial report:
Access: b29839 | Faculty: Faculty of Computing and IT | Time: Thu Oct  9 16:09:01 2025 | Credits: 0 | Balance UGX 0.00

Registering CSC2105 (3 credits) — should succeed
Access: b29839 | Faculty: Faculty of Computing and IT | Time: Thu Oct  9 16:09:01 2025 | Credits: 3 | Balance UGX 150000.00

Attempting to register a large load that exceeds credits — should raise
Caught Exception: Cannot register BIGLOAD: exceeds max credit limit (24)

Attempting to register courses that push fee balance beyond threshold — should raise
Caught Exception: Fee balance would exceed allowed threshold; clear dues first

Paying part of the fees (100000)
{'timestamp': 'Thu Oct  9 16:09:01 2025', 'paid': 100000}
Access: b29839 | Faculty: Faculty of Computing and IT | Time: Thu Oct  9 16:09:01 2025 | Credits: 3 | Balance UGX 50000.00


QUESTION 3 : HOSTEL VISITOR AUDIT

Description brief

A digital visitor log for UCU hostels can improve accountability by recording the latest visitor for each student room, making it easier to verify who visited, when, and for what reason. This reduces unauthorized entries, helps with contact tracing, and provides an auditable timestamped record for security personnel. Storing only the latest visitor (as requested) keeps the interface lightweight and privacy‑friendly while still providing timely accountability.

In [6]:
# Question 3 — Code Implementation
# Implement a class that stores only the latest visitor entry in a dictionary (no lists/tuples)
import time
import re

class HostelVisitorAudit:
    #Stores only the latest visitor entry for each student.
    #The internal storage is a dictionary mapping StudentID -> visit_record.
    
    def __init__(self, hostel_name):
        self.hostel_name = hostel_name
        # private dictionary: student_id -> record dict
        self.__latest_entries = {}

    def _validate_name(self, name):
        # Names should contain only letters and spaces (allowing common separators)
        if not isinstance(name, str) or not name.strip():
            return False
        # Regex: only letters (any case) and spaces
        return bool(re.fullmatch(r"[A-Za-z ]+", name.strip()))

    def record(self, student_id, visitor_name, reason=''):
        #Record a new visitor as the latest entry for the student.
        #Raises ValueError if validations fail.
        
        # Validate student_id (simple alphanumeric check)
        if not isinstance(student_id, str) or not student_id.strip():
            raise ValueError('Student ID must be a non-empty string')
        if not self._validate_name(visitor_name):
            raise ValueError('Visitor name must contain only letters and spaces')
        entry = {
            'visitor_name': visitor_name.strip(),
            'reason': reason,
            'timestamp': time.ctime()
        }
        self.__latest_entries[student_id.strip()] = entry
        return entry

    def update(self, student_id, visitor_name=None, reason=None):
        #Update the latest entry for a student. Only provided fields are updated.
        #Raises KeyError if student has no record.
        
        sid = student_id.strip()
        if sid not in self.__latest_entries:
            raise KeyError('No visitor record for this Student ID')
        if visitor_name is not None:
            if not self._validate_name(visitor_name):
                raise ValueError('Visitor name must contain only letters and spaces')
            self.__latest_entries[sid]['visitor_name'] = visitor_name.strip()
        if reason is not None:
            self.__latest_entries[sid]['reason'] = reason
        # refresh timestamp to indicate update time
        self.__latest_entries[sid]['timestamp'] = time.ctime()
        return self.__latest_entries[sid]

    def show_line(self, student_id):
        #Return a formatted audit line for the student.
        #Raises KeyError if no record.
        
        sid = student_id.strip()
        if sid not in self.__latest_entries:
            raise KeyError('No visitor record for this Student ID')
        rec = self.__latest_entries[sid]
        # formatted line
        return f"StudentID: {sid} | Hostel: {self.hostel_name} | Visitor: {rec['visitor_name']} | Reason: {rec['reason']} | Time: {rec['timestamp']}"

# Demonstration and exception handling
audit = HostelVisitorAudit('UCU Main Hostel')
print('Recording valid visitor...')
print(audit.record('b29839', 'John Blaq', reason='Study group'))

print('\nShowing line:')
print(audit.show_line('b29839'))

print('\nAttempting invalid name (with digits) — should raise and be handled')
try:
    audit.record('b30000', 'Mary Nabitaka', reason='Visit')
except Exception as e:
    print('Caught Exception:', e)

print('\nUpdating existing entry with valid name...')
print(audit.update('b29839', visitor_name='Jane', reason='Change of plans'))
print(audit.show_line('b29839'))


Recording valid visitor...
{'visitor_name': 'John Blaq', 'reason': 'Study group', 'timestamp': 'Thu Oct  9 16:09:18 2025'}

Showing line:
StudentID: b29839 | Hostel: UCU Main Hostel | Visitor: John Blaq | Reason: Study group | Time: Thu Oct  9 16:09:18 2025

Attempting invalid name (with digits) — should raise and be handled

Updating existing entry with valid name...
{'visitor_name': 'Jane', 'reason': 'Change of plans', 'timestamp': 'Thu Oct  9 16:09:18 2025'}
StudentID: b29839 | Hostel: UCU Main Hostel | Visitor: Jane | Reason: Change of plans | Time: Thu Oct  9 16:09:18 2025


QUESTION 4

CHOSEN SYSTEM: Mobile Money

Mobile money services like MTN Mobile Money, Airtel Money are used daily. Encapsulation helps maintain trust by ensuring confidential account credentials and balances are stored privately within a secure module/class. Only validated operations (deposit, withdrawal, transfer) are exposed, and each operation enforces checks like daily limits, minimum float, and authentication. This prevents direct tampering with balances and enforces transaction rules.


In [4]:
# Question 4 — Code Implementation
# Two classes interacting via encapsulation. One stores private confidential data (Wallet)
# The other (MerchantTerminal) interacts externally without breaking encapsulation.
import time

class Wallet:
    #Represents a user's mobile money wallet with private balance and PIN.
    #Only exposes safe methods for deposit, withdraw, and balance inquiries with
    #validations such as daily limit and minimum float.
    
    def __init__(self, phone_number, pin, initial_balance=0.0):
        self.phone_number = phone_number
        # private attributes
        self.__pin = str(pin)
        self.__balance = float(initial_balance)
        self.__daily_spent = 0.0
        self.__last_reset = time.strftime('%Y-%m-%d')
        # district-specific rules
        self.__daily_limit = 1_000_000.0  # max UGX per day
        self.__min_float = 500.0  # minimum balance to keep in wallet

    def __reset_daily_if_needed(self):
        today = time.strftime('%Y-%m-%d')
        if today != self.__last_reset:
            self.__daily_spent = 0.0
            self.__last_reset = today

    def check_pin(self, pin):
        return str(pin) == self.__pin

    def deposit(self, amount):
        if amount <= 0:
            raise ValueError('Deposit amount must be positive')
        self.__balance += float(amount)
        return {'timestamp': time.ctime(), 'deposit': amount}

    def withdraw(self, amount, pin):
        self.__reset_daily_if_needed()
        if not self.check_pin(pin):
            raise ValueError('Invalid PIN')
        if amount <= 0:
            raise ValueError('Withdrawal must be positive')
        if self.__daily_spent + amount > self.__daily_limit:
            raise ValueError('Daily withdrawal limit exceeded')
        if self.__balance - amount < self.__min_float:
            raise ValueError('Cannot withdraw: minimum float requirement')
        self.__balance -= float(amount)
        self.__daily_spent += float(amount)
        return {'timestamp': time.ctime(), 'withdrawn': amount}

    def get_balance_safe(self):
        # Expose only a safe view (no direct reference to private attribute name)
        return round(self.__balance, 2)

class MerchantTerminal:
    #Interacts with Wallets to request payments without accessing private data.
    #The terminal calls safe methods on Wallet and never touches internal fields.
    
    def __init__(self, terminal_id):
        self.terminal_id = terminal_id

    def request_payment(self, wallet, amount, pin):
        # The terminal requests a withdrawal; it does not read or set balance directly.
        try:
            result = wallet.withdraw(amount, pin)
            return {'status': 'success', 'detail': result}
        except Exception as e:
            return {'status': 'failed', 'reason': str(e)}

# Demonstration: show that external code cannot directly modify internal balance
user_wallet = Wallet('0700123456', pin='1234', initial_balance=10000)
terminal = MerchantTerminal('TERM001')
print('Initial safe balance:', user_wallet.get_balance_safe())

print('\nAttempting payment of UGX 2000 with correct PIN:')
print(terminal.request_payment(user_wallet, 2000, '1234'))
print('Balance after payment:', user_wallet.get_balance_safe())

print('\nAttempting to tamper: trying to set user_wallet.__balance directly...')
try:
    user_wallet.__balance = 9999999  # this creates a new attribute, does not modify private one
    print('Direct set created attribute __balance on object')
except Exception as e:
    print('Error during tampering attempt:', e)
print('Balance after tampering attempt (safe):', user_wallet.get_balance_safe())

print('\nAttempt withdrawal exceeding daily limit:')
print(terminal.request_payment(user_wallet, 2_000_000, '1234'))


Initial safe balance: 10000.0

Attempting payment of UGX 2000 with correct PIN:
{'status': 'success', 'detail': {'timestamp': 'Wed Oct  8 20:36:48 2025', 'withdrawn': 2000}}
Balance after payment: 8000.0

Attempting to tamper: trying to set user_wallet.__balance directly...
Direct set created attribute __balance on object
Balance after tampering attempt (safe): 8000.0

Attempt withdrawal exceeding daily limit:
{'status': 'failed', 'reason': 'Daily withdrawal limit exceeded'}
